In [1]:
from bqplot import Tooltip, AlbersUSA, OrdinalColorScale, ColorAxis, ColorScale
import pandas as pd
import numpy as np
from bqplot import pyplot as plt

In [35]:
import requests
URL = 'https://covidtracking.com/api/states'
r = requests.get(URL)
import json
covid = pd.read_json(json.dumps(r.json()))
covid.index = covid['state']
deaths = covid['death']
tests = covid['totalTestResults']
positive = covid['positive']

# merge deaths with FIPS

FIPS = pd.read_csv('states.csv')
FIPS.index = FIPS['USPS']
FIPS
FIPS['deaths'] = deaths
FIPS['tests'] = tests
FIPS['positive'] = positive

# create a series of populations
# create a column of normalized deaths

populations = pd.read_excel('state-populations.xlsx', index_col='State')
populations = populations['population']
FIPS['populations'] = populations
FIPS['normalized_deaths'] = FIPS['deaths']/FIPS['populations']
FIPS['normalized_tests'] = FIPS['tests']/FIPS['populations']
FIPS['case_fatality_ratio'] = FIPS['deaths']/FIPS['positive']
FIPS['positive_ratio'] = FIPS['positive'] / FIPS['tests']
FIPS['normalized_cases'] = FIPS['positive'] / FIPS['populations']
FIPS.head()
#covid.head()

,Name,FIPS,USPS,deaths,tests,positive,populations,normalized_deaths,normalized_tests,case_fatality_ratio,positive_ratio,normalized_cases
USPS,,,,,,,,,,,,
AL,Alabama,1,AL,3.0,4823,639,4903185,6.118472e-07,0.000984,0.004695,0.132490,0.000130
AK,Alaska,2,AK,2.0,2921,85,731545,2.733940e-06,0.003993,0.023529,0.029100,0.000116
AZ,Arizona,4,AZ,13.0,8193,738,7278717,1.786029e-06,0.001126,0.017615,0.090077,0.000101
AR,Arkansas,5,AR,3.0,3269,386,3017804,9.941003e-07,0.001083,0.007772,0.118079,0.000128
CA,California,6,CA,78.0,21259,3879,39512223,1.974073e-06,0.000538,0.020108,0.182464,0.000098


In [45]:
# with widget

import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa
import bqplot as bq
import ipywidgets as widgets
from IPython.display import display

def update_plot(*args):
    color_data = dict(zip(FIPS['FIPS'], FIPS[dropdown.value]))
        
    sc_cl = bqs.ColorScale(scheme='viridis')
    axis = bqa.ColorAxis(scale=sc_cl, num_ticks=5)
    map_styles = {'color': color_data,
              'scales': {'projection': sc_geo, 'color': sc_cl}, 
              'colors': {'default_color': 'Grey'}}
    states_map = bqm.Map(map_data=bqm.topo_load('map_data/USStatesMap.json'), **map_styles)
    fig.marks = [states_map]
    fig.axes = [axis]
    
dropdown = widgets.Dropdown(options=['normalized_deaths', 'deaths', 
                                     'tests', 'normalized_tests', 
                                     'case_fatality_ratio', 'positive_ratio',
                                     'normalized_cases', 'positive'])
dropdown.observe(update_plot, 'value')



color_data = dict(zip(FIPS['FIPS'], FIPS['normalized_deaths'])) 
sc_cl = bqs.ColorScale(scheme='viridis')
sc_geo = bqs.AlbersUSA(scale_factor=1200, translate=(500,400))
# TODO: better axis ticks
axis = bqa.ColorAxis(scale=sc_cl, tick_values=[0, 1e-5, 2e-5])
axis = bqa.ColorAxis(scale=sc_cl, num_ticks=5)
map_styles = {'color': color_data,
              'scales': {'projection': sc_geo, 'color': sc_cl}, 
              'colors': {'default_color': 'Grey'}}
states_map = bqm.Map(map_data=bqm.topo_load('map_data/USStatesMap.json'), **map_styles)

fig = bq.Figure(marks=[states_map], axes=[axis], title='COVID Metrics By State')

box = widgets.VBox([fig, dropdown])

display(box)

In [3]:
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa
import bqplot as bq

color_data = dict(zip(FIPS['FIPS'], FIPS['normalized_deaths'])) 
sc_cl = bqs.ColorScale(scheme='viridis')
sc_geo = bqs.AlbersUSA(scale_factor=1200, translate=(500,400))
axis = bqa.ColorAxis(scale=sc_cl)
map_styles = {'color': color_data,
              'scales': {'projection': sc_geo, 'color': sc_cl}, 
              'colors': {'default_color': 'Grey'}}
states_map = bqm.Map(map_data=bqm.topo_load('map_data/USStatesMap.json'), **map_styles)
bq.Figure(marks=[states_map], axes=[axis], title='US States Map Example')

Figure(axes=[ColorAxis(scale=ColorScale(scheme='viridis'))], fig_margin={'top': 60, 'bottom': 60, 'left': 60, …

In [ ]:
# could do a table or something here with a hover
def respond_to_hover(self, target):
    print(target)
states_map.on_hover(respond_to_hover)


In [23]:
# why does this require a captcha?
URL = 'http://datausa.io/api/data?drilldowns=State&measures=Population&year=latest'
URL = 'https://datausa.io/api/data?drilldowns=Nation&measures=Population&year=latest'
import requests
r = requests.get(URL)
r.content

b'<!DOCTYPE html>\n<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->\n<head>\n<title>Attention Required! | Cloudflare</title>\n\n<meta charset="UTF-8" />\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1" />\n<meta name="robots" content="noindex, nofollow" />\n<meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1" />\n<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/cf.errors.css" type="text/css" media="screen,projection" />\n<!--[if lt IE 9]><link rel="stylesheet" id=\'cf_styles-ie-css\' href="/cdn-cgi/styles/cf.errors.ie.css" type="text/css" media="screen,projection" /><![endif]-->\n<style type="text/css">body{ma